In [2]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "TexteFiscExactCode"

print(COUNTRY_NAME)

BFA Burkina Faso


In [5]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


In [6]:
from llm_setup import *

# Get legal documents from existing paperless Postgres

In [7]:
df, documents = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### Get the definitions of the Textes Fiscaux Standards and Textes Fiscaux Exactes from Postgres

In [8]:
df_fisc_exacts = get_postgress_data(sql_files['get_textes_fiscaux_exacts'], db='Ferdi')
df_fisc_exacts['TexteFiscExactCode'] = df_fisc_exacts['TexteFiscExactCode'].apply(lambda x: x[1:])
df_fisc_exacts['TexteFiscStandardCode'] = df_fisc_exacts['TexteFiscStandardCode'].apply(lambda x: x[1:])

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
df_fisc_standards = get_postgress_data(sql_files['get_textes_fiscaux_standards'], db='Ferdi')
df_fisc_standards['TexteFiscStandardCode'] = df_fisc_standards['TexteFiscStandardCode'].apply(lambda x: x[1:])

definitions = df_fisc_standards.set_index('TexteFiscStandardCode')['TexteFiscStandardComplet'].to_dict()
definitions

{'Gen_CGI': 'Code général des impôts. Le Code général des impôts est un recueil législatif centralisant toutes les lois et règlements relatifs à la fiscalité. Il définit les types d’impôts, les modalités de calcul, de déclaration et de paiement, ainsi que les obligations des contribuables et les pouvoirs de l’administration fiscale, assurant une gestion cohérente des impôts. Inclut les fonds de logement.',
 'Gen_IT': "Impôt sur le revenu. L'Impôt sur le revenu encadre la taxation des revenus des individus et des entreprises. Il précise les sources de revenus imposables, les barèmes d'imposition, les mécanismes de déclaration et de paiement, ainsi que les dispositions relatives aux exonérations et déductions fiscales.",
 'Gen_CGT': "Impôt sur les plus-values. L'Impôt sur les plus-values régit la taxation des gains réalisés lors de la vente d’actifs tels que biens immobiliers, actions ou autres investissements. Il définit les méthodes de calcul, les obligations de déclaration et de paiem

# Set the embeddings model and create a vectorized DB for the extracted texts

In [17]:
embed_model = get_voyage_ai_embedding_model()

In [18]:
vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embed_model, chunk_size = int(1024))

loading from local


In [19]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina)'

# Set the llm
the llm will be used to augment the retrieved data from the vector_index

In [20]:
llm = get_open_ai_llm()

# RAG to extract the Texte Fiscaux Exacte
 - create a vector engine using the above created vector database and the title of the document
 - using the llm to augment the retrieved response, and generate the response according to the prompt
 - will first extract the Texte Fiscaux Standard then use this to select the sub category

In [21]:
from difflib import get_close_matches

def find_most_similar_string(target, string_list):
    # Using difflib's get_close_matches function to find the closest match
    # It returns a list of close matches; we take the first (most similar) one
    closest_matches = get_close_matches(target, string_list, n=1, cutoff=0.0)
    return closest_matches[0] if closest_matches else None

def get_large_categ(vector_index, doc_name, llm, definitions=definitions):
    vector_query_engine = create_vector_engine(vector_index, doc_name, llm)

    """
    You are a helpful assistant, with legal document analysis expertise. Please analyze the given context.
    """
    response = vector_query_engine.query(
        f"""
        Consider the contents of the document with this title '{doc_name}'. Then, these definitions: {str(definitions)} 
        In which of these categories : {', '.join(list(definitions.keys()))}, is it part of ? Answer with a valid category, keep the category only. 
        Do not write an introduction, summary, rewrite or anything after the category.
        Do not include any "=" in the final answer. 
        If the document does not match any category, then reply with 'None'.
        If the result is none of the above categories, find the most suitable one, using the definitions and the context.
        """
    )

    print(doc_name, len(response.source_nodes))
    print([node.score for node in response.source_nodes])
    print([node.text for node in response.source_nodes])

    large_cat = response.response
    print(large_cat)

    if large_cat[-1] == "_":
        large_cat = large_cat[-1]

    definitions2 = df_fisc_exacts[df_fisc_exacts['TexteFiscStandardCode']==large_cat].set_index(['TexteFiscExactCode'])['TexteFiscExactComplet'].to_dict()
    print(definitions2.keys())

    if not definitions2:
        return "no_texte_fiscale_standard"
    else:
        response2 = vector_query_engine.query(
            f"""
            The following represents a list of definitions, where we define categories, followed by '=' and then the definitions: {", ".join([f"{k}={v}" for k,v in definitions2.items()])}
            Consider the contents of the document with this name '{doc_name}'.
            In which category, is it part of ? Answer with a valid category, keep the category only. Do not include any "=" in the final answer. 
            Do not include any "=" in the final answer. 
            If the document does not match any category, then reply with 'None'.
            If the result is none of the above categories, find the most suitable one, using the definitions and the context.
            """
        )

        print([node.score for node in response2.source_nodes])
        print([node for node in response2.source_nodes])
        print(response2.response)

    if not response2.response or response2.response == "None":
        return response.response
    else:
        if response2.response not in definitions2.keys():
            most_similar = find_most_similar_string(response2.response, definitions2.keys())
            return most_similar
        else:
            return response2.response

### Test the above function

In [22]:
response = get_large_categ(
    vector_index=vector_index,
    doc_name = doc_titles[0],
    llm=llm)

print(response)

Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina) 2
[0.7265625, 0.70703125]
[";\n- une copie du contrat de bail dûment enregistré ou de la quittance de renouvellement de\nbail.\n\n\n\nLe bénéfice de l’exonération, constatée par décision du Directeur général des impôts, prend\neffet à compter de la date d’introduction de la demande.\n\n\n\n(Le reste sans changement).\n\n\n\n\nArticle 28 :\n\nPour compter du 1er janvier 2009, l’article 13 1) de la loi n°014-2006/AN du 09 mai 2006\nportant détermination des ressources et des charges des collectivités territoriales au Burkina\nFaso est modifié et rédigé ainsi qu’il suit :\n\n\n\nArticle 13 1) nouveau :\n\nLes impôts et taxes ci-après recouvrés sur le territoire de la commune sont répartis entre la\ncommune et la région :\n\n\n\n- la contribution des patentes ;\n- la taxe de résidence ;\n- la taxe des biens de mainmorte ;\n- la taxe sur les armes ;\n- la taxe de jouissance ;\n- la contribution du secteur informel.\n\n\n\n\nNono

In [23]:
large_categs = list()

for doc in doc_titles:
    try:
        categ = get_large_categ(vector_index=vector_index, doc_name = doc, llm = llm)
    except Exception as e:
        print(e)

        categ = "non_determined"

    large_categs.append((doc, categ))

    

Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina) 2
[0.7265625, 0.70703125]
[";\n- une copie du contrat de bail dûment enregistré ou de la quittance de renouvellement de\nbail.\n\n\n\nLe bénéfice de l’exonération, constatée par décision du Directeur général des impôts, prend\neffet à compter de la date d’introduction de la demande.\n\n\n\n(Le reste sans changement).\n\n\n\n\nArticle 28 :\n\nPour compter du 1er janvier 2009, l’article 13 1) de la loi n°014-2006/AN du 09 mai 2006\nportant détermination des ressources et des charges des collectivités territoriales au Burkina\nFaso est modifié et rédigé ainsi qu’il suit :\n\n\n\nArticle 13 1) nouveau :\n\nLes impôts et taxes ci-après recouvrés sur le territoire de la commune sont répartis entre la\ncommune et la région :\n\n\n\n- la contribution des patentes ;\n- la taxe de résidence ;\n- la taxe des biens de mainmorte ;\n- la taxe sur les armes ;\n- la taxe de jouissance ;\n- la contribution du secteur informel.\n\n\n\n\nNono

In [28]:
df_TexteFiscExactCode = pd.DataFrame.from_dict(dict(large_categs), orient='index', columns=[table_column_name])

df_TexteFiscExactCode.to_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv", index=True)

df_TexteFiscExactCode

,TexteFiscExactCode
Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina),Gen_LF_LF
img_Décret n°2023-1307 (06.10.2023) Emplois miniers locaux 2023 (LinkedIn),Min_CM
Loi n°2017-021 (09.05.2017) Loi de finances rectificative 2017 (Assemblée Nationale),Gen_LF_LFR
Loi n°2013-032 (17.10.2013) Loi de finances rectificative 2013 (Droit-Afrique),Gen_LF_LFR
Loi n°2010-005 (29.01.2010) Modification CGIbis 2010 (DGI),Gen_FI_FImodif
...,...
Loi n°1993-037 (10.12.1993) Loi de finances 1994 (LégiBurkina),Gen_LF_LF
Décret n°2010-075 (03.03.2010) Taxes et redevances minières 2010 (Faolex),Min_CM_CMfisc
Loi n°2003-031 (08.05.2003) Code minier 2003 (Droit-Afrique),Min_CM_CM
img_Décret n°2010-524 (17.09.2010) Application Code des investissements 2010 (eRegulations Burkina Faso),Invest_CI_CIappli


## Load existing categories

In [25]:
# COUNTRY_NAME = 'BEN Bénin'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['Unnamed: 0']).to_dict().values())[0].items())

## Update column in table

In [26]:
from postgres_connection import bulk_update_textes

In [27]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    COUNTRY_NAME=COUNTRY_NAME)

Error updating Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina): (psycopg2.errors.ForeignKeyViolation) insert or update on table "textes" violates foreign key constraint "textes_TexteFiscExactCode_fkey"
DETAIL:  Key (TexteFiscExactCode)=(Gen_LF_LF) is not present in table "textes_fiscaux_exacts".

[SQL: UPDATE textes SET "TexteFiscExactCode"=%(TexteFiscExactCode)s WHERE textes."TexteCode" = %(textes_TexteCode)s]
[parameters: {'TexteFiscExactCode': 'Gen_LF_LF', 'textes_TexteCode': 'Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina)'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Error updating img_Décret n°2023-1307 (06.10.2023) Emplois miniers locaux 2023 (LinkedIn): (psycopg2.errors.ForeignKeyViolation) insert or update on table "textes" violates foreign key constraint "textes_TexteFiscExactCode_fkey"
DETAIL:  Key (TexteFiscExactCode)=(Min_CM) is not present in table "textes_fiscaux_exacts".

[SQL: UPDATE textes SET "TexteFiscExactCode"=%(Text